<a href="https://colab.research.google.com/github/kweenkeen/ReachOutandRead/blob/main/find_a_site.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geojson
!pip install geopandas
import pandas as pd
from datetime import date
import zipfile
import json
from geojson import Feature, FeatureCollection, Point
import geopandas
from google.colab import files

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.0 MB/s 
     |████████████████████████████████| 16.7 MB 219 kB/s 
     |████████████████████████████████| 6.3 MB 31.3 MB/s 


In [2]:
# UPLOAD SITE DATA REPORT

site_data_report = files.upload()

Saving MyROR Site Data Report 2022-07-05.xlsx to MyROR Site Data Report 2022-07-05.xlsx


In [3]:
filename = next(iter(site_data_report))
df = pd.read_excel(filename, converters={'Site ID': str})

# Select high level active only sites
high_level_active = ['Active', 'Active - Non-Reporting', 'Slated for deactivation', 'COVID-19 Freeze'] 
rslt_df = df[df['Status'].isin(high_level_active)] 

In [4]:
# UPLOAD AFFILIATE CONTACT LIST

affiliate_contact_list = files.upload()

Saving 2022.06.07 Find a Site Affiliate Contact Information.xlsx to 2022.06.07 Find a Site Affiliate Contact Information.xlsx


In [5]:
affiliate_filename = next(iter(affiliate_contact_list))

affiliate_contact_df = pd.read_excel(affiliate_filename)

rslt_df = rslt_df.join(affiliate_contact_df.set_index('Affiliate Name'), on='Affiliate Name')
rslt_df = rslt_df.set_index('Site ID')

contact_list1 = []
contact_list2 = []
contact_list3 = []
contact_list4 = []
affiliate_list1 = []
affiliate_list2 = []
affiliate_list3 = []
affiliate_list4 = []
affiliate_list1_city = []
affiliate_list2_city = []
affiliate_list3_city = []
affiliate_list4_city = []
affiliate_list1_state = []
affiliate_list2_state = []
affiliate_list3_state = []
affiliate_list4_state = []
affiliate_list1_latitude = []
affiliate_list2_latitude = []
affiliate_list3_latitude = []
affiliate_list4_latitude = []
affiliate_list1_longitude = []
affiliate_list2_longitude = []
affiliate_list3_longitude = []
affiliate_list4_longitude = []

for ind, row in rslt_df.iterrows():
  email_check1 = rslt_df['Primary Contact Email'][ind]
  email_check2 = rslt_df['Medical Consultant Email'][ind]
  email_check3 = rslt_df['Affiliate Contact'][ind]

  # If there is NO  Primary Contact Email, substitue Medical Consultant Email for Primary Contact Email
  if type(email_check1) == float and type(email_check2) == str:
    rslt_df.loc[ind, "Primary Contact Email"] = rslt_df.loc[ind, "Medical Consultant Email"]
    contact_list1.append(ind)
    affiliate_list1.append(rslt_df.loc[ind, "Affiliate Name"])
    affiliate_list1_city.append(rslt_df.loc[ind, "Physical Location: City"])   
    affiliate_list1_state.append(rslt_df.loc[ind, "Physical Location: State"]) 
    affiliate_list1_latitude.append(rslt_df.loc[ind, "Latitude"])
    affiliate_list1_longitude.append(rslt_df.loc[ind, "Longitude"])
  
  if type(email_check1) == str and type(email_check2) == float:
    contact_list4.append(ind)
    affiliate_list4.append(rslt_df.loc[ind, "Affiliate Name"])
    affiliate_list4_city.append(rslt_df.loc[ind, "Physical Location: City"])   
    affiliate_list4_state.append(rslt_df.loc[ind, "Physical Location: State"]) 
    affiliate_list4_latitude.append(rslt_df.loc[ind, "Latitude"])
    affiliate_list4_longitude.append(rslt_df.loc[ind, "Longitude"])

  # If there is NO Primary Contcat Email AND NO medical contact email, substitute info@reachoutandread.org
  elif type(email_check1) == float and type(email_check2) == float and type(email_check3) ==str:
    rslt_df.loc[ind, "Primary Contact Email"] = "info@reachoutandread.org"
    contact_list2.append(ind)
    affiliate_list2.append(rslt_df.loc[ind, "Affiliate Name"])
    affiliate_list2_city.append(rslt_df.loc[ind, "Physical Location: City"])   
    affiliate_list2_state.append(rslt_df.loc[ind, "Physical Location: State"]) 
    affiliate_list2_latitude.append(rslt_df.loc[ind, "Latitude"])
    affiliate_list2_longitude.append(rslt_df.loc[ind, "Longitude"])
  # If there is NO Primary Contact Email AND NO medical contact email AND NO affiliate contact email, substitute info@reachoutandread.org
  elif type(email_check1) == float and type(email_check2) == float and ((type(email_check3) ==float) or email_check3 == "NONE"):
    rslt_df.loc[ind, "Primary Contact Email"] = "info@reachoutandread.org"
    contact_list3.append(ind)
    affiliate_list3.append(rslt_df.loc[ind, "Affiliate Name"])
    affiliate_list3_city.append(rslt_df.loc[ind, "Physical Location: City"])   
    affiliate_list3_state.append(rslt_df.loc[ind, "Physical Location: State"]) 
    affiliate_list3_latitude.append(rslt_df.loc[ind, "Latitude"])
    affiliate_list3_longitude.append(rslt_df.loc[ind, "Longitude"])

#CHECK WHY DO THIS
rslt_df = rslt_df.reset_index(level=['Site ID'])

# Cut out columns you don't need
rslt_df = rslt_df[['Site ID', 'Site Name', 'Affiliate Contact','Physical Location: Line 1', 'Physical Location: Line 2', 'Physical Location: Line 3', 'Physical Location: City', 'Physical Location: State', 'Physical Location: Zip', 'Latitude', 'Longitude', 'Site Interest: potential volunteers', 'Site Interest: new or used book donations', 'Primary Contact Email']]

# Rename columns
rslt_df = rslt_df.rename(columns = {'Site ID': 'site_id', 'Site Name': 'sitename', 'Affiliate Contact': 'affiliate_email','Physical Location: Line 1': 'physical_1', 'Physical Location: Line 2':'physical_2', 'Physical Location: Line 3': 'physical_3', 'Physical Location: City': 'city', 'Physical Location: State' : 'state', 'Physical Location: Zip': 'zip', 'Site Interest: potential volunteers': 'site_volunteers', 'Site Interest: new or used book donations': 'site_donations', 'Primary Contact Email': 'primary_email'})
rslt_df = rslt_df.round({'Latitude': 4, 'Longitude': 4})


for ind, row in rslt_df.iterrows():
  if type(rslt_df.loc[ind, "site_volunteers"]) == float:
    rslt_df.loc[ind, "site_volunteers"] = "N"
  
  if type(rslt_df.loc[ind, "site_donations"]) == float:
    rslt_df.loc[ind, "site_donations"] = "N"

if (rslt_df['primary_email'].isna().any()) == True:
  print("Error with emails")
else: print("All is well with emails!")

if (rslt_df['site_volunteers'].isna().any()) == True:
  print("Error with Site Volunteers")
else:
  print("All is well with volunteers!")

if (rslt_df['site_donations'].isna().any()) == True:
  print("Error with Site Donations")
else:
  print("All is well with donations!")

# CREATE EXCEL SPREADSHEET WITH MISSING CONTACT INFO

contact_df1 = pd.DataFrame({'Missing Primary Contact': pd.Series(contact_list1), 'Affiliate Name': pd.Series(affiliate_list1), 'City': pd.Series(affiliate_list1_city), 'State': pd.Series(affiliate_list1_state), 'Latitude': pd.Series(affiliate_list1_latitude), 'Longitude': pd.Series(affiliate_list1_longitude)})
contact_df2 = pd.DataFrame({'Missing Primary Contact and Medical Contact': pd.Series(contact_list2), 'Affiliate Name': pd.Series(affiliate_list2), 'City': pd.Series(affiliate_list2_city), 'State': pd.Series(affiliate_list2_state), 'Latitude': pd.Series(affiliate_list2_latitude), 'Longitude': pd.Series(affiliate_list2_longitude)})
contact_df3 = pd.DataFrame({'Missing Primary Contact, Medical Contact, and Affiliate Contact': pd.Series(contact_list3), 'Affiliate Name': pd.Series(affiliate_list3), 'City': pd.Series(affiliate_list3_city), 'State': pd.Series(affiliate_list3_state), 'Latitude': pd.Series(affiliate_list3_latitude), 'Longitude': pd.Series(affiliate_list3_longitude)})
contact_df4 = pd.DataFrame({'Missing Medical Contact': pd.Series(contact_list4), 'Affiliate Name': pd.Series(affiliate_list4), 'City': pd.Series(affiliate_list4_city), 'State': pd.Series(affiliate_list4_state), 'Latitude': pd.Series(affiliate_list4_latitude), 'Longitude': pd.Series(affiliate_list4_longitude)})

contact_df1 = contact_df1.sort_values(by=['Affiliate Name', 'City'])
contact_df2 = contact_df2.sort_values(by=['Affiliate Name', 'City'])
contact_df3 = contact_df3.sort_values(by=['Affiliate Name', 'City'])
contact_df4 = contact_df4.sort_values(by=['Affiliate Name', 'City'])

today = str(date.today())

contact_filename = 'Missing_Contact_List_' + today + '.xlsx'
# writer_contact = pd.ExcelWriter(contact_filename, engine='xlsxwriter')

with pd.ExcelWriter(contact_filename) as writer: 
  contact_df1.to_excel(writer, sheet_name ="Missing Primary Contact", index = False)
  contact_df4.to_excel(writer, sheet_name ="Missing Medical Contact", index = False)
  contact_df2.to_excel(writer, sheet_name ="Missing Primary and Medical", index = False)
  contact_df3.to_excel(writer, sheet_name ="Missing Primary, Med, Affiliate", index = False)

# SAVE RESULTS DATAFRAME AS EXCEL FILE 
today = str(date.today())
excel_filename = "find-a-site_" + today + ".xlsx"
rslt_df.to_excel(excel_filename, index=False)

#CREATE CSV OF SITES WITHOUT GPS COORDINATES

nan_df = rslt_df[rslt_df['Latitude'].isna()]
nan_csv = "find-a-site_" + today + "_nan" + ".csv"
nan_df.to_csv(nan_csv,encoding='utf-8-sig', index=False)

# CREATE CSV UTF-8 FILE FOR "SMALL" JSON FILE
small_df = rslt_df[['site_id', 'Latitude', 'Longitude']]
small_df = small_df.dropna()
small_csv = "find-a-site_" + today + "_small" + ".csv"
small_df.to_csv(small_csv,encoding='utf-8-sig', index=False)

# CREATE GEOJSON FILE
geo_small_df = geopandas.GeoDataFrame(
    small_df, geometry=geopandas.points_from_xy(small_df['Longitude'], small_df['Latitude']))
geo_small_df.to_json()
geojson_filename =  "find-a-site_" + today + "_small_.geojson.txt"
geo_small_df.to_file(geojson_filename, driver="GeoJSON") 

# Fill in NaNs with blank
rslt_df = rslt_df.fillna("")

# CREATE CSV UTF-8 FILE FOR "LARGE" JSON FILE
large_csv = "find-a-site_" + today + "_large" + ".csv"
rslt_df.to_csv(large_csv,encoding='utf-8-sig', index=False)

# CREATE JSON.TXT FILE
json_dict = rslt_df.to_json(orient="records")
parsed = json.loads(json_dict)
#json_object = json.dumps(parsed)
json_object = json.dumps(parsed, indent=2, sort_keys=True)
json_filename = "find-a-site_" + today + "_large_.json.txt"
with open(json_filename, "w") as outfile:
    outfile.write(json_object)

# ZIP FILES TOGETHER INTO FOLDER
list_files = [contact_filename, excel_filename, large_csv, nan_csv, small_csv, json_filename, geojson_filename]
# list_files = [contact_filename, json_filename, geojson_filename]

folder_name = "Find-A-Site_" + today + ".zip"
with zipfile.ZipFile(folder_name, 'w') as zipF:
  for file in list_files:
    zipF.write(file, compress_type=zipfile.ZIP_DEFLATED)

files.download(folder_name)
  


All is well with emails!
All is well with volunteers!
All is well with donations!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
small_df.isnull().values.any()

False

In [ ]:
print(contact_filename)

Missing_Contact_List_2022-07-01.xlsx
